<a href="https://colab.research.google.com/github/rajiv1977/AI/blob/main/CNNTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# TEST DEV
```



In [19]:
# Mount your Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [21]:
class ImageProcessingSettings:
    """Struct to hold settings for image processing."""

    def __init__(self, convert_to_gray=False, resize=False, resize_width=1, resize_height=1, rotate=False, num_classes=10):
        """
        Initialize settings for image processing.

        Args:
            convert_to_gray (bool): Flag to convert images to grayscale.
            resize (bool): Flag to indicate if resizing is needed.
            resize_width (int): Downscale factor for width.
            resize_height (int): Downscale factor for height.
            rotate (bool): Flag to indicate if rotation is needed.
            num_classes (int): Number of classes or features.
        """
        self.convert_to_gray = convert_to_gray
        self.resize = resize
        self.resize_width = resize_width  # New attribute for resize width
        self.resize_height = resize_height  # New attribute for resize height
        self.rotate = rotate
        self.num_classes = num_classes

In [22]:
def load_images_from_folder(folder, settings):
    """
    Load images from specified folder and its subfolders.

    Args:
        folder (str): Path to the main dataset folder.
        settings (ImageProcessingSettings): Settings for image processing.

    Returns:
        np.array: Array of loaded images.
        np.array: Array of labels corresponding to the images.
        list: List of label names.
    """
    images = []
    labels = []
    label_names = []

    # Get subfolder names (class labels)
    subfolders = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]

    # Check if any subfolders were found
    if not subfolders:
        print("Error: No subfolders found in the dataset path.")
        return np.array([]), np.array([]), []

    label_names = subfolders
    num_classes = len(label_names)

    # Iterate through subfolders and load images
    for label_index, subfolder in enumerate(subfolders):
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            try:
                # Load image with appropriate color mode
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE if settings.convert_to_gray else cv2.IMREAD_COLOR)

                # Check if image loading was successful
                if img is None:
                    print(f"Warning: Could not load image: {img_path}")
                    continue

                # Resize image if specified
                if settings.resize:
                    img = cv2.resize(img, (settings.resize_width, settings.resize_height))

                # Add channel dimension for grayscale images if needed
                if settings.convert_to_gray and len(img.shape) == 2:
                    img = img.reshape(img.shape[0], img.shape[1], 1)

                images.append(img)
                labels.append(label_index)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")

    return np.array(images), np.array(labels), label_names


In [23]:
def build_cnn_model(input_shape, num_classes):
    """
    Build a Convolutional Neural Network (CNN) model using Keras.

    Args:
        input_shape (tuple): Shape of the input images.
        num_classes (int): Number of output classes.

    Returns:
        keras.Model: Compiled CNN model.
    """
    model = models.Sequential()

    # Input layer: (height, width, channels)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same')) # reduced kernel size, added padding
    model.add(layers.Dropout(0.5))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same')) # reduced kernel size, added padding
    model.add(layers.Dropout(0.5))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten layer
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))  # Use softmax for multi-class classification

    return model

In [ ]:
def main():
    # Parameters
    epochs = 25
    batch_size = 32
    learning_rate = 0.01  # Not used in this example but included for completeness

    # Set the dataset path (update this to your dataset path)
    dataset_path = "/content/drive/My Drive/AI_Outputs/TrainAndValidateData"

    # Verify if the dataset path is valid
    if not os.path.isdir(dataset_path):
        print(f"Error: Invalid dataset path: {dataset_path}")
        print("Please ensure the path is correct and the directory exists.")
        return  # Exit the program if the path is invalid

    # Get a list of subfolders and count them
    subfolders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]
    num_classes = len(subfolders)
    print(f"Found {num_classes} subfolders in the dataset path.")

    # Create image processing settings
    settings = ImageProcessingSettings(
        convert_to_gray=True,
        resize=True,
        resize_width=40,
        resize_height=40,
        num_classes=num_classes
    )

    # Load images and labels
    images, labels, label_names = load_images_from_folder(dataset_path, settings)
    labels = to_categorical(labels, settings.num_classes)

    print("Number of images loaded:", len(images))
    print("Shape of first image:", images[0].shape)

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.3, random_state=1)  # Changed test_size to 0.3

    # Print the shapes of X_train and y_train
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)

    # Get the actual input shape from the loaded images
    input_shape = images[0].shape

    # Build the model, now passing the actual input shape
    model = build_cnn_model(input_shape, settings.num_classes)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    # ... (your existing code) ...

    # Validate the model
    val_loss, val_accuracy = model.evaluate(X_val, y_val)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

    # Predict on the validation set
    y_pred = model.predict(X_val)

    # Convert predictions and true labels to class indices
    predicted_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(y_val, axis=1)

    # Print predicted and true y values for the first few samples
    num_samples_to_print = 10  # Adjust as needed
    for i in range(num_samples_to_print):
        print(f"Sample {i + 1}: Predicted Class: {predicted_classes[i]}, True Class: {true_classes[i]}")

if __name__ == "__main__":
    main()

Found 10 subfolders in the dataset path.
Number of images loaded: 8789
Shape of first image: (40, 40, 1)
X_train shape: (6152, 40, 40, 1)
y_train shape: (6152, 10)
Epoch 1/25
193/193 ━━━━━━━━━━━━━━━━━━━━ 40s 201ms/step - accuracy: 0.1856 - loss: 16.7422 - val_accuracy: 0.4983 - val_loss: 1.5245
Epoch 2/25
193/193 ━━━━━━━━━━━━━━━━━━━━ 39s 190ms/step - accuracy: 0.4265 - loss: 1.6414 - val_accuracy: 0.5461 - val_loss: 1.4054
Epoch 3/25
 77/193 ━━━━━━━━━━━━━━━━━━━━ 19s 167ms/step - accuracy: 0.5046 - loss: 1.4041